# OpenFOAM Example with DAPI
## Accessing OpenFOAM (Version 6 or Version 7) using DesignSafe API (DAPI)

## Install DesignSafe API (DAPI)

In [ ]:
# DAPI installation
!pip uninstall dapi --yes --quiet
!pip install dapi --user --quiet

# Uncomment to install development version
# !pip install git+https://github.com/DesignSafe-CI/dapi.git@dev --user --quiet

# Uncomment to install editable local version
# !pip install -e ../

## Import DAPI and Initialize Client

In [ ]:
import os
import json
from datetime import datetime

# Import DAPI components
from dapi import (
    DSClient,
    SubmittedJob,
    interpret_job_status,
    AppDiscoveryError,
    FileOperationError,
    JobSubmissionError,
    SystemInfoError,
    JobMonitorError,
    STATUS_TIMEOUT,
    STATUS_UNKNOWN,
    TAPIS_TERMINAL_STATES,
)

print("DAPI imports successful.")

In [ ]:
# Initialize DAPI client
try:
    print("Initializing DSClient...")
    ds = DSClient()
    print("DSClient initialized successfully.")
except Exception as e:
    print(f"Initialization failed: {e}")
    raise SystemExit("Stopping notebook due to client initialization failure.")

## Discover OpenFOAM Application

In [ ]:
# Find OpenFOAM applications
try:
    print("Searching for OpenFOAM applications...")
    openfoam_apps = ds.apps.find("openfoam", verbose=True)

    if not openfoam_apps:
        print("No OpenFOAM applications found.")
        raise SystemExit("No OpenFOAM apps available.")

except Exception as e:
    print(f"Error finding OpenFOAM apps: {e}")
    raise SystemExit("Failed to discover OpenFOAM applications.")

In [ ]:
# Get details for OpenFOAM application
app_id = "openfoam-7.0u4"  # Use the same app as the original example

try:
    print(f"Getting details for app: {app_id}")
    app_details = ds.apps.get_details(app_id, verbose=True)

    if not app_details:
        raise SystemExit(f"Could not find details for app '{app_id}'.")

    print(f"\nApp Description: {app_details.description}")
    print(f"App Version: {app_details.version}")
    print(f"Execution System: {app_details.jobAttributes.execSystemId}")

except Exception as e:
    print(f"Error getting app details: {e}")
    raise SystemExit("Failed to get OpenFOAM app details.")

## Illustrative case
### Simulation of the wind flow around a square cross-section of the building using RANS model
>Case directory: DH1_run contains $0$, *constant* and *system* directories.

![frame.png](attachment:frame.png)

## Mesh generation using the *blockMesh* utility 
> Note: The grid resolution in the plot doesn't meet the requirement for running RANS simulations. This is only for an illustrative case.

![Mesh.png](attachment:Mesh.png)

In [ ]:
# Display blockMeshDict file content (if available locally)
try:
    with open("DH1_run/system/blockMeshDict", "r") as f:
        file_contents = f.read()
        print(file_contents)
except FileNotFoundError:
    print(
        "blockMeshDict file not found locally. Please ensure the case directory is available."
    )

In [ ]:
# Display velocity boundary conditions (if available locally)
try:
    with open("DH1_run/0/U", "r") as f:
        file_contents = f.read()
        print(file_contents)
except FileNotFoundError:
    print(
        "Velocity file (U) not found locally. Please ensure the case directory is available."
    )

In [ ]:
# Display turbulence properties (if available locally)
try:
    with open("DH1_run/constant/turbulenceProperties", "r") as f:
        file_contents = f.read()
        print(file_contents)
except FileNotFoundError:
    print(
        "turbulenceProperties file not found locally. Please ensure the case directory is available."
    )

## Setup the OpenFOAM run configuration
(Reference:
Harish, Ajay Bangalore; Govindjee, Sanjay; McKenna, Frank (2020) "CFD Notebooks (Beginner)." DesignSafe-CI. https://doi.org/10.17603/ds2-w2x6-nm09.)

> Specify the number of nodes and processors for parallel computing <br>
> Select a solver <br>
> Change to your input directory

In [ ]:
# Configuration parameters
ds_path = "/MyData/OpenFOAM_examples/DH1_run"  # Update with your actual path
max_job_minutes = 120  # 2 hours
tacc_allocation = "ASC25049"  # Update with your allocation

# Job configuration
job_name = "OpenFOAM-DAPI-Demo"
node_count = 1
cores_per_node = 2
solver = "pisoFoam"
mesh_option = "On"
decomp_option = "On"

In [ ]:
# Translate local path to Tapis URI
try:
    print(f"Translating path: {ds_path}")
    input_uri = ds.files.translate_path_to_uri(ds_path, verify_exists=True)
    print(f"Input Directory Tapis URI: {input_uri}")
except FileOperationError as e:
    print(f"Error translating/verifying path '{ds_path}': {e}")
    print(
        "Please update the ds_path variable with the correct path to your OpenFOAM case directory."
    )
    raise SystemExit("Stopping due to path error.")
except Exception as e:
    print(f"Unexpected error: {e}")
    raise SystemExit("Stopping due to unexpected error.")

## Generate Job Request

In [ ]:
# Generate job request using DAPI
try:
    print("\nGenerating job request dictionary...")
    job_dict = ds.jobs.generate_request(
        app_id=app_id,
        input_dir_uri=input_uri,
        max_minutes=max_job_minutes,
        allocation=tacc_allocation,
        job_name=job_name,
    )

    # Customize job parameters for OpenFOAM
    job_dict["nodeCount"] = node_count
    job_dict["coresPerNode"] = cores_per_node

    # Add OpenFOAM-specific parameters
    if "parameterSet" not in job_dict:
        job_dict["parameterSet"] = {}
    if "appArgs" not in job_dict["parameterSet"]:
        job_dict["parameterSet"]["appArgs"] = []

    # Add OpenFOAM parameters (mesh, solver, decomp)
    openfoam_params = [
        {"name": "mesh", "arg": mesh_option},
        {"name": "solver", "arg": solver},
        {"name": "decomp", "arg": decomp_option},
    ]

    job_dict["parameterSet"]["appArgs"].extend(openfoam_params)

    print("\n--- Generated Job Request Dictionary ---")
    print(json.dumps(job_dict, indent=2, default=str))
    print("---------------------------------------")

except (AppDiscoveryError, ValueError, JobSubmissionError) as e:
    print(f"Error generating job request: {e}")
    raise SystemExit("Stopping due to job request generation error.")
except Exception as e:
    print(f"Unexpected error during job request generation: {e}")
    raise SystemExit("Stopping due to unexpected generation error.")

## Submit Job to TACC

In [ ]:
# Submit the job
try:
    print("\nSubmitting the job request...")
    submitted_job = ds.jobs.submit_request(job_dict)
    print(f"Job Submitted Successfully!")
    print(f"Job UUID: {submitted_job.uuid}")

except JobSubmissionError as e:
    print(f"Job submission failed: {e}")
    print("\n--- Failed Job Request ---")
    print(json.dumps(job_dict, indent=2, default=str))
    print("--------------------------")
    raise SystemExit("Stopping due to job submission error.")
except Exception as e:
    print(f"Unexpected error during job submission: {e}")
    raise SystemExit("Stopping due to unexpected submission error.")

## Monitor Job Status

In [ ]:
# Monitor job execution
if "submitted_job" not in locals():
    print("Error: submitted_job not found.")
    raise SystemExit("Stopping notebook.")

# Monitor the job with 30-second intervals
try:
    print("\nMonitoring job execution...")
    final_status = submitted_job.monitor(interval=30)
    print(f"\nJob {submitted_job.uuid} monitoring finished.")
    print(f"Final status: {final_status}")

except Exception as e:
    print(f"Error during job monitoring: {e}")
    # Continue even if monitoring fails - we can check status manually

## Check Job Status and Results

In [ ]:
# Interpret job outcome
try:
    print("\n--- Job Outcome ---")
    ds.jobs.interpret_status(final_status, submitted_job.uuid)
    print("-------------------")
except Exception as e:
    print(f"Error interpreting job status: {e}")

In [ ]:
# Display runtime summary for completed jobs
if final_status in ["FINISHED", "FAILED"]:
    print(f"\nAttempting to display runtime summary...")
    try:
        submitted_job.print_runtime_summary(verbose=False)
    except Exception as e:
        print(f"Could not display runtime summary: {e}")
else:
    print(f"\nSkipping runtime summary because job ended with status: {final_status}.")

In [ ]:
# Get current job status
try:
    current_status = ds.jobs.get_status(submitted_job.uuid)
    print(f"\nCurrent status of job {submitted_job.uuid}: {current_status}")
except JobMonitorError as e:
    print(f"Error getting job status: {e}")
except Exception as e:
    print(f"Unexpected error occurred: {e}")

## Access Job Output and Archive

In [ ]:
# Display job output if in terminal state
if "submitted_job" in locals() and final_status in submitted_job.TERMINAL_STATES:
    print(f"\n--- Job Output for {submitted_job.uuid} (Status: {final_status}) ---")
    max_output_lines = 50

    # Get standard output
    try:
        stdout_content = submitted_job.get_output_content(
            "tapisjob.out", max_lines=max_output_lines, missing_ok=False
        )
        if stdout_content is not None:
            print(f"\n--- Last {max_output_lines} lines of tapisjob.out ---")
            print(stdout_content)
            print("------------------------------------")
        else:
            print("\n[INFO] tapisjob.out was not found or is empty.")
    except FileOperationError as e:
        print(f"\n[ERROR] Could not retrieve tapisjob.out: {e}")
    except Exception as e:
        print(f"\n[ERROR] Unexpected error retrieving tapisjob.out: {e}")

    # Get error output for failed jobs
    if final_status in ["FAILED", "ARCHIVING_FAILED"]:
        try:
            stderr_content = submitted_job.get_output_content(
                "tapisjob.err", max_lines=max_output_lines, missing_ok=True
            )
            if stderr_content is not None:
                print(f"\n--- Last {max_output_lines} lines of tapisjob.err ---")
                print(stderr_content)
                print("------------------------------------")
            else:
                print("\n[INFO] tapisjob.err was not found.")
        except FileOperationError as e:
            print(f"\n[ERROR] Could not retrieve tapisjob.err: {e}")
        except Exception as e:
            print(f"\n[ERROR] Unexpected error retrieving tapisjob.err: {e}")
    print("----------------------------------------------------")
else:
    print("\nSkipping job output display (job not in terminal state).")

In [ ]:
# Access job archive
try:
    print(f"\nAttempting to access archive information...")
    archive_uri = submitted_job.archive_uri
    if archive_uri:
        print(f"Job Archive Tapis URI: {archive_uri}")
        print("\nListing archive contents (root):")
        outputs = ds.files.list(archive_uri)
        if outputs:
            for item in outputs:
                print(
                    f"- {item.name} (Type: {item.type}, Size: {item.size} bytes, Modified: {item.lastModified})"
                )
        else:
            print("No files found in the archive root directory.")
    else:
        print("Archive URI not available for this job.")
except FileOperationError as e:
    print(f"Could not list archive files: {e}")
except Exception as e:
    print(f"Unexpected error while accessing archive information: {e}")

## Post-processing
### Using *ParaView* to visualize the flow fields
*ParaView* can read *OpenFOAM* files using *.foam*.

>If .foam is not included in the case directory, you can copy the archive results:

>**import shutil**

>**shutil.copy2('case_directory/foam.foam', 'destination_path')**

![ParaView.png](attachment:ParaView.png)

### Plot the time series of the force coefficients
> Update the path to point to your job's archive directory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Example code for processing force coefficients
# Update the path based on your job's archive location
try:
    # You would need to download or access the specific results file
    # from the job archive. This is just an example structure.

    # For now, we'll show the structure that would be used:
    print("To plot force coefficients:")
    print("1. Access the job archive using the archive_uri")
    print("2. Download the postProcessing/forceCoeffs/0/forceCoeffs.dat file")
    print("3. Use the plotting code below with the actual data")

    # Example plotting code (commented out since we don't have actual data)
    """
    def is_float(string):
        try:
            return float(string)
        except ValueError:
            return False

    data = []
    with open('forceCoeffs.dat', 'r') as f:
        d = f.readlines()
        for i in d:
            k = i.rstrip().split("\t")
            data.append([float(i) if is_float(i) else i for i in k]) 

    data = np.array(data[9:], dtype='O')
    
    # Plot drag coefficient
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.plot(data[100:,0], data[100:,2])
    plt.xlabel('Time')
    plt.ylabel('$C_d$')
    plt.title('Drag Coefficient vs Time')
    
    # Plot lift coefficient
    plt.subplot(2, 1, 2)
    plt.plot(data[100:,0], data[100:,3])
    plt.xlabel('Time')
    plt.ylabel('$C_l$')
    plt.title('Lift Coefficient vs Time')
    
    plt.tight_layout()
    plt.show()
    """

except Exception as e:
    print(f"Note: Actual post-processing would require accessing the job results: {e}")

## Summary

This notebook demonstrates how to:
1. **Initialize DAPI client** with authentication
2. **Discover OpenFOAM applications** using `ds.apps.find()`
3. **Get application details** using `ds.apps.get_details()`
4. **Generate job requests** using `ds.jobs.generate_request()`
5. **Submit jobs** using `ds.jobs.submit_request()`
6. **Monitor job execution** using `submitted_job.monitor()`
7. **Access job outputs and archives** using DAPI file operations

Key differences from the Agave version:
- Uses DAPI's modern authentication system
- Leverages DAPI's built-in job monitoring and status interpretation
- Provides better error handling and user feedback
- Uses Tapis v3 APIs through DAPI's simplified interface